In [1]:
from transformato import FreeEnergyCalculator

import transformato
import pytest
import sys, os
import logging
import shutil
import pathlib
from transformato import load_config_yaml, SystemStructure, ProposeMutationRoute, IntermediateStateFactory
import parmed as pm
import copy
import numpy as np
# read in specific topology with parameters
from parmed.charmm.parameters import CharmmParameterSet
import subprocess

for conf in ['../config/ethane-ethanol-solvation-free-energy.yaml']:
    configuration = load_config_yaml(config=conf,
                    input_dir='../data/', output_dir='../data/')

    # load systems
    s1 = SystemStructure(configuration, 'structure1')
    s2 = SystemStructure(configuration, 'structure2')
    a = ProposeMutationRoute(s1, s2)

    # generate mutation route
    mutation_list = a.generate_mutations_to_common_core_for_mol1(nr_of_steps_for_el=5, nr_of_steps_for_bonded_parameters=5)
    # write intermediate states for systems
    i = IntermediateStateFactory(system=s1, mutation_list=mutation_list, configuration=configuration)
    i.generate_intermediate_states()
    paths = pathlib.Path(i.path).glob('**/*.sh')
    for path in sorted(paths):
        run_dir = path.parent
        # because path is object not string
        print(f"Start sampling for: {path}")
        print(f"In directory: {run_dir}")
        try:
            exe = subprocess.run(['bash', str(path), str(run_dir)], check=True, capture_output=True,text=True)
        except TypeError:
            exe = subprocess.run(['bash', str(path), str(run_dir)], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(exe.stdout)
        print('Capture stderr')
        print(exe.stderr)

    # generate mutation route
    mutation_list = a.generate_mutations_to_common_core_for_mol2(nr_of_steps_for_el=5)
    # write intermediate states
    i = IntermediateStateFactory(system=s2, mutation_list=mutation_list, configuration=configuration)
    i.generate_intermediate_states()

    paths = pathlib.Path(i.path).glob('**/*.sh')
    for path in sorted(paths):
        run_dir = path.parent
        # because path is object not string
        print(f"Start sampling for: {path}")
        print(f"In directory: {run_dir}")
        try:
            exe = subprocess.run(['bash', str(path), str(run_dir)], check=True, capture_output=True,text=True)
        except TypeError:
            exe = subprocess.run(['bash', str(path), str(run_dir)], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(exe.stdout)
        print('Capture stderr')
        print(exe.stderr)

    f= FreeEnergyCalculator(configuration, 'ethane')
    f.load_trajs(thinning=1)
    f.calculate_dG_to_common_core()
    ddG, dddG = f.end_state_free_energy_difference
    print(f"Free energy difference: {ddG}")
    print(f"Uncertanty: {dddG}")
    #assert(ddG == 10.0)

    f.show_summary()

    f= FreeEnergyCalculator(configuration, 'ethanol')
    f.load_trajs(thinning=1)
    f.calculate_dG_to_common_core()

    f.show_summary()
    #shutil.rmtree(pathlib.Path(i.path).parent)


RDKit WARNING: [16:02:05] Enabling RDKit 2019.09.2 jupyter extensions
/home/mwieder/anaconda3/envs/transformato/lib/python3.7/site-packages/transformato-0.0.0+99.g028cb55.dirty-py3.7.egg/transformato/utils.py:17: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  settingsMap = yaml.load(stream)
/home/mwieder/anaconda3/envs/transformato/lib/python3.7/site-packages/parmed/charmm/parameters.py:905: UserWarning: LONEPAIR type COLINEAR not supported; only BISEctor and RELAtive supported
  'RELAtive supported' % words[1])
/home/mwieder/anaconda3/envs/transformato/lib/python3.7/site-packages/parmed/charmm/parameters.py:832: UserWarning: No charge for WEI3
  warnings.warn('No charge for %s' % resname)
/home/mwieder/anaconda3/envs/transformato/lib/python3.7/site-packages/parmed/charmm/parameters.py:867: UserWarning: WARNING: Ignoring "DELETE ACCE NE2" because entity type ACCE not use

/home/mwieder/Work/Projects/transformato/data/ethane/waterbox/lig/lig.sdf


OSError: Bad input file /home/mwieder/Work/Projects/transformato/data/ethane/waterbox/lig/lig.sdf

In [8]:
from itertools import product

In [9]:
for a in product(['a', 'b', 'c'], ['x', 'y']):
    print(a)

('a', 'x')
('a', 'y')
('b', 'x')
('b', 'y')
('c', 'x')
('c', 'y')
